In [35]:
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

In [10]:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title
sleep(5)
elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
sleep(5)
driver.close()

In [11]:
url = "https://www.amazon.fr/livre-africain/s?k=livre+africain"
driver = webdriver.Firefox()
driver.get(url)
sleep(10)
html_code = driver.content
driver.close()

In [12]:
source = BeautifulSoup(html_code, "html.parser")

In [15]:
articles = source.find_all("div", attrs={"data-component-type":"s-search-result"})
print(len(articles))
print(type(articles[0]))

60
<class 'bs4.element.Tag'>


In [42]:
url = "https://www.amazon.fr/livre-africain/s?k=livre+africain"
driver = webdriver.Firefox()
driver.get(url)
sleep(2)
# Accept cookies
acc = driver.find_element(By.ID, "sp-cc-accept").click()

elems = driver.find_elements(By.CSS_SELECTOR, "div.s-result-item.s-asin")
# elems = driver.find_elements(By.CSS_SELECTOR, "div.s-product-image-container > span")
articles = []
for elm in elems:
    ActionChains(driver).key_down(Keys.CONTROL).click(elm).key_up(Keys.CONTROL).perform()
    driver.switch_to.window(driver.window_handles[1])
    
    # elm.click()
    sleep(1)
    title = driver.find_element(By.ID, "productTitle").text.strip()
    price = driver.find_element(By.ID, "corePriceDisplay_desktop_feature_div").text.strip()
    note = driver.find_element(By.XPATH, '//*[@id="acrPopover"]/span[1]/a/span').text.strip()
    thumbnail = driver.find_element(By.ID, "landingImage").get_attribute("src")
    description = driver.find_element(By.ID, "bookDescription_feature_div").text.strip()
    articles.append({
        "title": title, 
        "price": price, 
        "thumbnail": thumbnail, 
        "description": description, 
        "instock": True, 
        "note": note
    })
    driver.close()
driver.close()

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.performActions@chrome://remote/content/marionette/driver.sys.mjs:1458:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [44]:
url = "https://www.amazon.fr/livre-africain/s?k=livre+africain"
driver = webdriver.Firefox()
driver.get(url)

for i in range(0, 60):
    sleep(2)
    # Accept cookies
    acc = driver.find_element(By.ID, "sp-cc-accept").click()

    xpath = f'//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[{i+3}]'
    driver.find_element(By.XPATH, xpath).click()

    sleep(1)
    title = driver.find_element(By.ID, "productTitle").text.strip()
    price = driver.find_element(By.ID, "corePriceDisplay_desktop_feature_div").text.strip()
    note = driver.find_element(By.XPATH, '//*[@id="acrPopover"]/span[1]/a/span').text.strip()
    thumbnail = driver.find_element(By.ID, "landingImage").get_attribute("src")
    description = driver.find_element(By.ID, "bookDescription_feature_div").text.strip()
    articles.append({
        "title": title, 
        "price": price, 
        "thumbnail": thumbnail, 
        "description": description, 
        "instock": True, 
        "note": note
    })

    driver.back()
driver.close()

WebDriverException: Message: Failed to decode response from marionette


In [ ]:
def extract_article_data(article: WebElement) -> dict:
    driver.click()
    title = driver.find_element(By.ID, "productTitle").text.strip()
    price = driver.find_element(By.ID, "corePriceDisplay_desktop_feature_div").text.strip()
    note = driver.find_element(By.XPATH, "//*[@id="acrPopover"]/span[1]/a/span").text.strip()
    thumbnail = driver.find_element(By.ID, "landingImage").get_attribute("src")
    description = driver.find_element(By.ID, "bookDescription_feature_div").text.strip()
    driver.back()
    return {
        "title": title, 
        "price": price, 
        "thumbnail": thumbnail, 
        "description": description, 
        "instock": True, 
        "note": note
    }

In [24]:
print(len(elems))
print(type(elems[0]))

60
<class 'selenium.webdriver.remote.webelement.WebElement'>


In [43]:
def get_description(description_url: str) -> str:
    base_url = "https://books.toscrape.com/catalogue/"
    description_html_code = requests.get(base_url+description_url).content
    description_source = BeautifulSoup(description_html_code, "html.parser")
    header = description_source.find(id="product_description")
    description = header.find_next_sibling("p")
    return description.get_text().strip()

def extract_article_data(article: Tag) -> dict:
    h3_a = article.h3.a
    price = article.find("p", class_="price_color").get_text().strip()
    title = h3_a.get("title")
    description_url = h3_a.get("href").split("../")[-1]
    description = get_description(description_url)
    return {"title":title, "price":price, "thumbnail":"", "description": description, "instock": True, "note": 5}

In [43]:
def get_description(description_url: str) -> str:
    base_url = "https://books.toscrape.com/catalogue/"
    description_html_code = requests.get(base_url+description_url).content
    description_source = BeautifulSoup(description_html_code, "html.parser")
    header = description_source.find(id="product_description")
    description = header.find_next_sibling("p")
    return description.get_text().strip()

def extract_article_data(article: Tag) -> dict:
    h3_a = article.h3.a
    price = article.find("p", class_="price_color").get_text().strip()
    title = h3_a.get("title")
    description_url = h3_a.get("href").split("../")[-1]
    description = get_description(description_url)
    return {"title":title, "price":price, "thumbnail":"", "description": description, "instock": True, "note": 5}

In [45]:
books = [extract_article_data(article) for article in articles]
print(books)

[{'title': 'Tipping the Velvet', 'price': '£53.74', 'thumbnail': '', 'description': '"Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s "Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s dresser and the two head for the bright lights of Leicester Square where they begin a glittering career as music-hall stars in an all-singing and dancing double ac

In [6]:
source.prettify

<bound method Tag.prettify of 
<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    Historical Fiction | 
     Books to Scrape - Sandbox

</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    
" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../../../../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../../../../sta